# Processing HDMA data
Downloaded 2019 LAR data from https://ffiec.cfpb.gov/data-publication/2019 

Note: in final dataset, index for race is 10, with 1=Black and index for gender is 11 with 1=Female


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [3]:
# set max columns to 100
pd.set_option('display.max_columns', 100)

## Basic preprocessing
* Drop columns that we will not use. Some columns (eg, interest_rate, rate_spread) are dropped because they are missing values for loans that were denied
* Iterate through dataframe in chunks (it's too large to do all at once)
* * Drop rows where race or sex is not available or 'joint' or where age is missing
* * Drop rows where income, conforming loan limit, combined loan to value ratio, debt to income ratio, or property value is NAN
* * Drop rows where business or commercial purpose, property value, or interest_only_payment is exempt
* * Limit data to these values because the vast majority of samples fall into these categories 
* * * conforming loan limit == C 
* * * derived dwelling category = Single family/site built
* * * loan type = 1 (conventional, not backed by FHA, VA, RHS or FSA)
* * * loan purpose = 2 (home purchase)
* * * site_built = 1
* Save each processed chunk as a CSV

In [102]:
columns_to_drop = ['activity_year', 'derived_ethnicity', 'applicant_credit_score_type', 'co_applicant_credit_score_type',
                    'applicant_ethnicity_1', 'applicant_ethnicity_2', 'applicant_ethnicity_3', 'applicant_ethnicity_4',
                    'applicant_ethnicity_5', 'co_applicant_ethnicity_1', 'co_applicant_ethnicity_2', 'co_applicant_ethnicity_3',
                    'co_applicant_ethnicity_4', 'co_applicant_ethnicity_5', 'applicant_ethnicity_observed', 
                    'co_applicant_ethnicity_observed', 'applicant_race_1', 'applicant_race_2', 'applicant_race_3', 'applicant_race_4',
                    'applicant_race_5', 'co_applicant_race_1', 'co_applicant_race_2', 'co_applicant_race_3', 'co_applicant_race_4',
                    'co_applicant_race_5', 'applicant_race_observed', 'co_applicant_race_observed', 'applicant_sex', 'co_applicant_sex',
                    'applicant_sex_observed', 'co_applicant_sex_observed', 'co_applicant_age', 'applicant_age_above_62', 
                    'co_applicant_age_above_62', 'denial_reason_1', 'denial_reason_2', 'denial_reason_3', 'denial_reason_4', 
                    'lei', 'county_code', 'derived_msa_md', 'census_tract', 'tract_population', 'tract_minority_population_percent',
                    'ffiec_msa_md_median_family_income', 'tract_owner_occupied_units', 
                    'tract_one_to_four_family_homes', 'tract_median_age_of_housing_units', 'tract_to_msa_income_percentage', 
                    'aus_1', 'aus_2', 'aus_3', 'aus_4', 'aus_5', 'total_loan_costs', 'total_points_and_fees', 'origination_charges',
                    'discount_points', 'lender_credits', 'multifamily_affordable_units', 'intro_rate_period', 'prepayment_penalty_term',
                    'submission_of_application', 'initially_payable_to_institution', 'manufactured_home_secured_property_type',
                    'manufactured_home_land_property_interest', 'derived_loan_product_type', 'purchaser_type',
                    'preapproval', 'lien_status', 'reverse_mortgage', 'open_end_line_of_credit', 'other_nonamortizing_features', 
                    'balloon_payment', 'negative_amortization', 'interest_only_payment', 'interest_rate', 'rate_spread']

In [103]:
chunksize = 10 ** 6
i=0
with pd.read_csv('2019_public_lar_one_year.csv', chunksize=chunksize, low_memory=False) as reader:
    for chunk in reader:
        chunk = chunk.drop(columns=columns_to_drop)
        chunk.head()

        chunk = chunk[chunk['derived_race'] != 'Race Not Available']
        chunk = chunk[chunk['derived_race'] != 'Joint']
        chunk = chunk[chunk['derived_sex'] != 'Sex Not Available']
        chunk = chunk[chunk['derived_sex'] != 'Joint']

        chunk = chunk.dropna(subset=['income', 'conforming_loan_limit', 'combined_loan_to_value_ratio',
                                        'debt_to_income_ratio', 'property_value', 'loan_term'])

        chunk = chunk[chunk['conforming_loan_limit'] == 'C']
        chunk = chunk[chunk['derived_dwelling_category'] == 'Single Family (1-4 Units):Site-Built']

        chunk = chunk[chunk.business_or_commercial_purpose != 1111]
        chunk = chunk[chunk.property_value != 'Exempt']
        chunk = chunk[chunk.debt_to_income_ratio != 'Exempt']
        chunk = chunk[chunk.applicant_age != '8888']

        # create a new column that is 0 if 'action_taken' is 1 or 2 and 1 if 'action_taken' is 3 and -1 otherwise
        chunk['action_taken'] = chunk['action_taken'].apply(lambda x: 0 if x == 1 or x == 2 else 1 if x == 3 else -1)
        chunk = chunk[chunk['action_taken'] != -1]
        

        chunk = chunk[chunk.loan_type == 1]
        chunk = chunk[chunk.loan_purpose == 1]
        chunk = chunk[chunk.construction_method == 1]
        chunk = chunk.drop(columns=['conforming_loan_limit', 'derived_dwelling_category', 'loan_type', 'loan_purpose', 'construction_method'])
        
        # print(type(chunk))
        chunk.to_csv('hmda' + str(i) + '.csv', index=False)
        i+=1
        


Now, load all pieces into a single dataframe - hopefully it will fit?

In [104]:
columns = chunk.columns

In [172]:
df = pd.DataFrame(columns=columns)
for i in range(18):
    df = pd.concat([df, pd.read_csv('hmda' + str(i) + '.csv', low_memory=False)])

## Encode categorical features

Columns to use if we need to further limit the data, but to delete before use
* state_code

Columns that are categorical and need to be integer-coded
* derived_race
* derived_sex
* action_taken (this is the label and is already binary, but right now 0 is accept, so switch that)
* business_or_commerial_purpose
* hoepa_status
* debt_to_income_ratio
* occupancy_type
* applicant_age

Columns that are already numerical 
* loan_amount
* combined_loan_to_value_ratio
* loan_term
* property_value
* total_units
* income

In [173]:
race_white = pd.Series([1 if x == 'White' else 0 for x in df.derived_race])
race_black = pd.Series([1 if x == 'Black or African American' else 0 for x in df.derived_race])
sex_female = pd.Series([1 if x == 'Female' else 0 for x in df.derived_sex])
label = pd.Series([1 if x == 0 else -1 for x in df.action_taken])
business_purpose = pd.Series([1 if x  == 1 else 0 for x in df.business_or_commercial_purpose])
hoepa = pd.Series([1 if x == 1 else 0 for x in df.hoepa_status]) # count NA as not a high-cost mortgage
not_principal_residence = pd.Series([0 if x == 1 else 1 for x in df.occupancy_type])


In [174]:
debt_to_income_mapping = {'<20%': 15, '20%-<30%' : 25, '30%-<36%' : 33, '36' : 36, '37' : 37, '38':38, '39':39,
                        '40': 40, '41':41, '42': 42, '43':43, '44':44, '45':45, '46':46, '47':47, '48':48, '49':49,
                        '50%-60%': 55, '>60%' : 65}
debt_to_income_rat = pd.Series([debt_to_income_mapping[x] for x in df.debt_to_income_ratio])


In [175]:
age_mapping = {'<25': 20, '25-34': 30, '35-44': 40, '45-54': 50, '55-64': 60, '65-74': 70, '>74': 80}
age = pd.Series([age_mapping[x] for x in df.applicant_age])

In [176]:
df = df.drop(columns=['derived_race', 'derived_sex', 'action_taken', 'business_or_commercial_purpose', 'hoepa_status', 
                         'occupancy_type', 'debt_to_income_ratio', 'applicant_age'])

df['business_purpose'] = business_purpose
df['hoepa'] = hoepa
df['not_principal_residence'] = not_principal_residence
df['debt_to_income_ratio'] = debt_to_income_rat
df['race_white'] = race_white
df['race_black'] = race_black
df['female'] = sex_female
df['age'] = age
df['label'] = label

In [177]:
df.to_csv('hmda_all.csv', index=False)

In [178]:
all = pd.read_csv('hmda_all.csv')

## Subsampling
We want the data to be approximately balanced across race (since we are not attempting to study the impact of size of minority groups on fairrness) and also across the label (since we are not focusing on learning under label imbalance).

Since the data skews heavily towards White people and approved loans, we will be limited by the availability of suitable data points. But, there are still plenty of data points to obtain a large-enough balanced dataset.

In [179]:
# subsample from the data to balance label and race
balanced_race = pd.concat([df[df.race_black == 1].sample(n=110000, random_state=1129), df[df.race_white == 1].sample(n=110000, random_state=1129)])

balanced_race = balanced_race.drop(columns=['race_white']) # duplicate info with race_black, now


In [182]:
result = pd.concat([balanced_race[balanced_race.label == 1].sample(n=20000, random_state=1129),
                balanced_race[balanced_race.label == -1].sample(n=20000, random_state=1129)])

In [183]:
result = result.drop(columns=['state_code'])

In [184]:
result.to_csv('hmda_balanced.csv', index=False)

In [4]:
data = pd.read_csv('hmda_balanced.csv')

## Train/test split
Train/test split the data into 10 folds to be able to provide averages across multiple random splits.

In [20]:
y = data['label'].to_numpy()
X = data.drop(columns=['label']).to_numpy()

num_splits = 10
dataset = 'hmda'
kf = KFold(n_splits=num_splits,shuffle=True,random_state=42)
i = 0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/(num_splits-1), random_state=42) #41 testing other

    #     scaler = StandardScaler().fit(X_train)
    #     X_train = scaler.transform(X_train)
    #     X_test = scaler.transform(X_test)
    #     X_val = scaler.transform(X_val)

    X_train = pd.DataFrame(X_train, columns = data.drop(columns=['label']).columns)
    X_test = pd.DataFrame(X_test, columns = data.drop(columns=['label']).columns)
    X_val = pd.DataFrame(X_val, columns = data.drop(columns=['label']).columns)

    y_train = pd.Series(y_train).reset_index(drop=True)
    y_test = pd.Series(y_test).reset_index(drop=True)
    y_val = pd.Series(y_val).reset_index(drop=True)

    X_train['label'] = y_train
    X_test['label'] = y_test
    X_val['label'] = y_val

    X_train.to_csv("./" + dataset + "_train" + str(i) + ".csv", index=False)
    X_test.to_csv("./" + dataset + "_test" + str(i) + ".csv", index=False)
    X_val.to_csv("./" + dataset + "_val" + str(i) + ".csv", index=False)

    i += 1

## Test basic models

In [187]:
# import log reg model
from sklearn.linear_model import LogisticRegression
# import decision tree
from sklearn.tree import DecisionTreeClassifier
# import random forest
from sklearn.ensemble import RandomForestClassifier
# import standard scalar
from sklearn.preprocessing import StandardScaler

In [209]:
lr_acc, dt_acc, rf_acc = [], [], []
for i in range(10):
    train = pd.read_csv('hmda_train' + str(i) +'.csv')
    test = pd.read_csv('hmda_test' + str(i) + '.csv')
    X_train = train.drop(columns=['label'])
    y_train = train.label
    X_test = test.drop(columns=['label'])
    y_test = test.label

    model = DecisionTreeClassifier(max_depth=10, min_samples_leaf=10)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    dt_acc.append(accuracy_score(y_test, y_pred))

    model = RandomForestClassifier(max_depth=3, n_estimators=100)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rf_acc.append(accuracy_score(y_test, y_pred))

    model = LogisticRegression(solver='saga', max_iter=1000)
    scalar = StandardScaler()
    X_train = scalar.fit_transform(X_train)
    X_test = scalar.transform(X_test)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    from sklearn.metrics import accuracy_score
    lr_acc.append(accuracy_score(y_test, y_pred))

In [210]:
sum(lr_acc)/len(lr_acc)

0.6222874999999999

In [211]:
sum(dt_acc)/len(dt_acc)

0.6541250000000001

In [212]:
sum(rf_acc)/len(rf_acc)

0.6451125

In [94]:
import numpy as np

In [95]:
ary = np.array(data)

In [98]:
# find correlationof ary
corr = np.corrcoef(ary, rowvar=False)

/Users/annameyer/miniforge3/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/annameyer/miniforge3/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [101]:
data.site_built.unique()

array([1])

In [165]:
y_train = [1 if x == 1 else -1 for x in y_train]
y_test = [1 if x == 1 else -1 for x in y_test]

In [204]:
prd = np.matmul(X_train.T, X_train)
pinv = np.linalg.pinv(prd - 1* np.identity(prd.shape[0]))
w = np.matmul(np.matmul(pinv, X_train.T), y_train)

In [205]:
w

array([ 0.00900511,  0.008506  ,  0.00838206,  0.00735305, -0.00274628,
       -0.00500112, -0.00715401,  0.00740463, -0.02463459, -0.22247714,
       -0.18733041,  0.02074614, -0.05510479])

In [206]:
preds = np.matmul(w, X_test.T)

In [207]:
preds = np.array([1 if x > 0 else -1 for x in preds])

In [208]:
# import accuracy
from sklearn.metrics import accuracy_score
# calculate accuracy
accuracy_score(y_test, preds)

0.61775

In [ ]:
23212 - B

In [9]:
len(data)

40000

In [18]:
white = data[data.race_black == 0]

In [22]:
len(white)

16788

23212 - Black (58.0%) so, White (42.0%)

20063 - Female (50.2%), so Male, (49.8%)

13056 - Black Female (32.6%), so, Black Male (25.4%)

7007 - White Female (17.5%), so, White Male (24.5%)

In [24]:
data_inc = pd.read_csv('../income/income.csv')

In [29]:
fem = data_inc[data_inc.sex_male == 0]

In [35]:
fem[fem.race_white==1]

,hours_worked_over40,sex_male,age_under35,age_35to55,age_over55,work_forprofit,work_nonprofit,work_gov,work_selfemploy,education_noHS,education_HS,education_college,rel_married,rel_divorced,rel_single,occ_business,occ_science,occ_social,occ_protective,occ_sales,occ_admin,occ_trades,occ_crafts,occ_logistics,birth_usstate,birth_usterr,birth_europe,birth_asia,birth_americas,birth_africa,role_hoh,role_child,role_family,role_housemate,role_institution,role_group,race_white,race_black,race_indigenous,race_asian,race_other,race_multi,label
1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
4,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
6,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
8,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
11,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32679,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
32680,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
32681,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
32683,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1


In [38]:
(844-428)/len(data_inc)

0.012725604160293667